# 

# Exploring Mental Health Data


## Keras - 1 Attempt

## Attempt 1

In [1]:
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC
import json
import time

# Reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Data Loading
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Infer id and target columns
id_col = 'id'
target_col = 'Depression'
# Map train target if necessary
if target_col not in train_df.columns and 'class' in train_df.columns:
    train_df[target_col] = train_df['class']

# Encode target
ename = LabelEncoder()
y_enc = ename.fit_transform(train_df[target_col].astype(str)).astype(int)

# Prepare features
X_train = train_df.drop(columns=[target_col, id_col], errors='ignore')
train_ids = train_df[id_col]

X_test = test_df.drop(columns=[target_col, id_col], errors='ignore')
test_ids = test_df[id_col]
y_val = None

# Feature engineering
# Drop all-missing columns
cols_all_missing = [col for col in X_train.columns if X_train[col].isna().all()]
X_train = X_train.drop(columns=cols_all_missing)
X_test = X_test.drop(columns=cols_all_missing)

# Identify categorical columns with cardinality <=50
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
low_card_cols = [col for col in cat_cols if X_train[col].nunique() <= 50]
# Numeric columns
num_cols = X_train.select_dtypes(include=['number']).columns.tolist()

# Preprocessing Pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, low_card_cols)
])

# Fit and transform data
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.transform(X_test)

# Model architecture
n_samples, n_features = X_train_proc.shape
# Two hidden layers for small dataset
layer1_units = min(n_features * 2, 128)
layer2_units = min(n_features, 64)

inputs = tf.keras.Input(shape=(n_features,))
x = tf.keras.layers.Dense(layer1_units, activation='relu')(inputs)
x = tf.keras.layers.Dropout(0.3)(x)
x = tf.keras.layers.Dense(layer2_units, activation='relu')(x)
x = tf.keras.layers.Dropout(0.3)(x)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(x)
model = tf.keras.Model(inputs=inputs, outputs=outputs)

model.compile(optimizer='adam',
              loss='binary_crossentropy',
              metrics=['accuracy', AUC(name='auc')])

# Callbacks
callbacks = [
    EarlyStopping(patience=10, restore_best_weights=True, verbose=1),
    ModelCheckpoint('model_checkpoint.h5', save_best_only=True, verbose=1)
]

# Training
start_time = time.time()
history = model.fit(
    X_train_proc, y_enc,
    validation_split=0.2,
    epochs=100,
    batch_size=128,
    callbacks=callbacks,
    verbose=2
)
duration = time.time() - start_time

# Logging results
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1],
    'training_duration_seconds': duration
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction and submission
raw_preds = model.predict(X_test_proc)
probs = raw_preds.flatten()
final_preds = (probs > 0.5).astype(int)

# Build submission
submission = pd.DataFrame({id_col: test_ids.reset_index(drop=True), target_col: final_preds})
submission.to_csv('submission_result.csv', index=False)

2025-07-10 12:28:57.975409: E external/local_xla/xla/stream_executor/cuda/cuda_fft.cc:467] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
E0000 00:00:1752150537.992814 1689569 cuda_dnn.cc:8579] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
E0000 00:00:1752150537.997935 1689569 cuda_blas.cc:1407] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
W0000 00:00:1752150538.012742 1689569 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752150538.012755 1689569 computation_placer.cc:177] computation placer already registered. Please check linkage and avoid linking the same target more than once.
W0000 00:00:1752150538.012757 1689569 computation_placer.cc:177] computation placer alr

Epoch 1/100


I0000 00:00:1752150542.736409 1689710 service.cc:152] XLA service 0x786b58009d30 initialized for platform CUDA (this does not guarantee that XLA will be used). Devices:
I0000 00:00:1752150542.736437 1689710 service.cc:160]   StreamExecutor device (0): NVIDIA A100-SXM4-40GB, Compute Capability 8.0
2025-07-10 12:29:02.801371: I tensorflow/compiler/mlir/tensorflow/utils/dump_mlir_util.cc:269] disabling MLIR crash reproducer, set env var `MLIR_CRASH_REPRODUCER_DIRECTORY` to enable.
I0000 00:00:1752150542.996289 1689710 cuda_dnn.cc:529] Loaded cuDNN version 90501
I0000 00:00:1752150544.376291 1689710 device_compiler.h:188] Compiled cluster using XLA!  This line is logged at most once for the lifetime of the process.
2025-07-10 12:29:09.757329: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_50', 120 bytes spill stores, 120 bytes spill loads

2025-07-10 12:29:09.855315: I external/


Epoch 1: val_loss improved from inf to 0.15206, saving model to model_checkpoint.h5


880/880 - 11s - 13ms/step - accuracy: 0.9286 - auc: 0.9644 - loss: 0.1779 - val_accuracy: 0.9399 - val_auc: 0.9749 - val_loss: 0.1521
Epoch 2/100

Epoch 2: val_loss improved from 0.15206 to 0.15199, saving model to model_checkpoint.h5


880/880 - 3s - 4ms/step - accuracy: 0.9364 - auc: 0.9705 - loss: 0.1610 - val_accuracy: 0.9398 - val_auc: 0.9750 - val_loss: 0.1520
Epoch 3/100

Epoch 3: val_loss improved from 0.15199 to 0.15121, saving model to model_checkpoint.h5


880/880 - 3s - 4ms/step - accuracy: 0.9369 - auc: 0.9710 - loss: 0.1598 - val_accuracy: 0.9399 - val_auc: 0.9752 - val_loss: 0.1512
Epoch 4/100

Epoch 4: val_loss did not improve from 0.15121
880/880 - 3s - 4ms/step - accuracy: 0.9373 - auc: 0.9716 - loss: 0.1582 - val_accuracy: 0.9401 - val_auc: 0.9751 - val_loss: 0.1514
Epoch 5/100

Epoch 5: val_loss did not improve from 0.15121
880/880 - 3s - 4ms/step - accuracy: 0.9377 - auc: 0.9716 - loss: 0.1580 - val_accuracy: 0.9395 - val_auc: 0.9751 - val_loss: 0.1516
Epoch 6/100

Epoch 6: val_loss did not improve from 0.15121
880/880 - 3s - 4ms/step - accuracy: 0.9374 - auc: 0.9718 - loss: 0.1575 - val_accuracy: 0.9399 - val_auc: 0.9751 - val_loss: 0.1517
Epoch 7/100

Epoch 7: val_loss did not improve from 0.15121
880/880 - 3s - 4ms/step - accuracy: 0.9377 - auc: 0.9720 - loss: 0.1573 - val_accuracy: 0.9397 - val_auc: 0.9752 - val_loss: 0.1517
Epoch 8/100

Epoch 8: val_loss did not improve from 0.15121
880/880 - 3s - 4ms/step - accuracy: 0.93

2025-07-10 12:29:51.889754: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16', 4 bytes spill stores, 4 bytes spill loads



2932/2932 ━━━━━━━━━━━━━━━━━━━━ 6s 2ms/step


In [ ]:
print(duration)

49.26095795631409


## Keras Tuner - 2 Attempts

## Attempt 1

In [ ]:
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC
import json
import time
import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

# Reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Data Loading
train_df = pd.read_csv('playground-series-s4e11/train.csv')
test_df = pd.read_csv('playground-series-s4e11/test.csv')

# Infer id and target columns
id_col = 'id'
target_col = 'Depression'
# Map train target if necessary
if target_col not in train_df.columns and 'class' in train_df.columns:
    train_df[target_col] = train_df['class']

# Encode target
ename = LabelEncoder()
y_enc = ename.fit_transform(train_df[target_col].astype(str)).astype(int)

# Prepare features
X_train = train_df.drop(columns=[target_col, id_col], errors='ignore')
train_ids = train_df[id_col]

X_test = test_df.drop(columns=[target_col, id_col], errors='ignore')
test_ids = test_df[id_col]
y_val = None

# Feature engineering
# Drop all-missing columns
cols_all_missing = [col for col in X_train.columns if X_train[col].isna().all()]
X_train = X_train.drop(columns=cols_all_missing)
X_test = X_test.drop(columns=cols_all_missing)

# Identify categorical columns with cardinality <=50
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
low_card_cols = [col for col in cat_cols if X_train[col].nunique() <= 50]
# Numeric columns
num_cols = X_train.select_dtypes(include=['number']).columns.tolist()

# Preprocessing Pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, low_card_cols)
])

# Fit and transform data
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.transform(X_test)

# Model architecture
n_samples, n_features = X_train_proc.shape

# Define early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Input dimension
n_features = X_train_proc.shape[1]

# HyperModel class
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = Dropout(drop)(x)
        outputs = Dense(1, activation='sigmoid')(x)
        model = Model(inputs, outputs)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', AUC(name='auc')])
        return model

# Tuner setup
bs = 32  # batch size
ep = 20  # epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=True,
    project_name='bayesian_tuner'
)

# Search for the best hyperparameters
if y_val is not None:
    tuner.search(
        X_train_proc, y_enc,
        validation_data=(X_val_proc, y_val),
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )
else:
    tuner.search(
        X_train_proc, y_enc,
        validation_split=0.2,
        batch_size=bs, epochs=ep,
        callbacks=[early_stopping, checkpoint]
    )

# Build the best model
model = tuner.hypermodel.build(tuner.get_best_hyperparameters(1)[0])

# Retrain the model with original callbacks and data
if y_val is not None:
    history = model.fit(
        X_train_proc, y_enc,
        validation_data=(X_val_proc, y_val),
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
else:
    history = model.fit(
        X_train_proc, y_enc,
        validation_split=0.2,
        epochs=100, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )

# Logging results
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1],
    'training_duration_seconds': duration
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction and submission
raw_preds = model.predict(X_test_proc)
probs = raw_preds.flatten()
final_preds = (probs > 0.5).astype(int)

# Build submission
submission = pd.DataFrame({id_col: test_ids.reset_index(drop=True), target_col: final_preds})
submission.to_csv('submission_result.csv', index=False)

Trial 1 Complete [00h 00m 11s]
val_loss: 0.15245185792446136

Best val_loss So Far: 0.15245185792446136
Total elapsed time: 00h 00m 11s


3518/3518 - 10s - 3ms/step - accuracy: 0.9324 - auc: 0.9662 - loss: 0.1724 - val_accuracy: 0.9386 - val_auc: 0.9741 - val_loss: 0.1531


NameError: name 'duration' is not defined

## Attempt 2

In [1]:
import random
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.impute import SimpleImputer
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import StandardScaler, OneHotEncoder, LabelEncoder
from sklearn.pipeline import Pipeline
import tensorflow as tf
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint
from tensorflow.keras.metrics import AUC
import json
import time
import keras_tuner as kt
from tensorflow.keras.layers import Input, Dense, Dropout
from tensorflow.keras.models import Model

# Reproducibility
random.seed(42)
np.random.seed(42)
tf.random.set_seed(42)

# Data Loading
train_df = pd.read_csv('train.csv')
test_df = pd.read_csv('test.csv')

# Infer id and target columns
id_col = 'id'
target_col = 'Depression'
# Map train target if necessary
if target_col not in train_df.columns and 'class' in train_df.columns:
    train_df[target_col] = train_df['class']

# Encode target
ename = LabelEncoder()
y_enc = ename.fit_transform(train_df[target_col].astype(str)).astype(int)

# Prepare features
X_train = train_df.drop(columns=[target_col, id_col], errors='ignore')
train_ids = train_df[id_col]

X_test = test_df.drop(columns=[target_col, id_col], errors='ignore')
test_ids = test_df[id_col]
y_val = None

# Feature engineering
# Drop all-missing columns
cols_all_missing = [col for col in X_train.columns if X_train[col].isna().all()]
X_train = X_train.drop(columns=cols_all_missing)
X_test = X_test.drop(columns=cols_all_missing)

# Identify categorical columns with cardinality <=50
cat_cols = X_train.select_dtypes(include=['object', 'category']).columns.tolist()
low_card_cols = [col for col in cat_cols if X_train[col].nunique() <= 50]
# Numeric columns
num_cols = X_train.select_dtypes(include=['number']).columns.tolist()

# Preprocessing Pipeline
numeric_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='median')),
    ('scaler', StandardScaler())
])
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore', sparse_output=False))
])
preprocessor = ColumnTransformer(transformers=[
    ('num', numeric_transformer, num_cols),
    ('cat', categorical_transformer, low_card_cols)
])

# Fit and transform data
X_train_proc = preprocessor.fit_transform(X_train)
X_test_proc = preprocessor.transform(X_test)

# Model architecture
n_samples, n_features = X_train_proc.shape

# Define early stopping and model checkpoint
early_stopping = EarlyStopping(monitor='val_loss', patience=10, restore_best_weights=True)
checkpoint = ModelCheckpoint('best_model.h5', monitor='val_loss', save_best_only=True)

# Input dimension
n_features = X_train_proc.shape[1]

# HyperModel class
class MyHyperModel(kt.HyperModel):
    def build(self, hp):
        layers = hp.Int('layers', 2, 8)
        units = hp.Int('units', 64, 1024)
        act = hp.Choice('activation', ['relu'])
        drop = hp.Float('dropout', 0.0, 0.5)
        opt = hp.Choice('optimizer', ['adam'])
        lr = hp.Float('learning_rate', 1e-5, 0.01, sampling='log')

        inputs = Input(shape=(n_features,))
        x = inputs
        for _ in range(layers):
            x = Dense(units, activation=act)(x)
            x = Dropout(drop)(x)
        outputs = Dense(1, activation='sigmoid')(x)
        model = Model(inputs, outputs)
        model.compile(optimizer=opt, loss='binary_crossentropy', metrics=['accuracy', AUC(name='auc')])
        return model

# Tuner setup
bs = 32  # batch size
ep = 20  # epochs

tuner = kt.BayesianOptimization(
    MyHyperModel(),
    objective='val_loss',
    max_trials=10,
    executions_per_trial=1,
    seed=42,
    overwrite=True,
    project_name='bayesian_tuner'
)

# Search for the best hyperparameters
if y_val is not None:
    tuner.search(
        X_train_proc, y_enc,
        validation_data=(X_val_proc, y_val),
        batch_size=bs, epochs=100,
        callbacks=[early_stopping, checkpoint]
    )
else:
    tuner.search(
        X_train_proc, y_enc,
        validation_split=0.2,
        batch_size=bs, epochs=100,
        callbacks=[early_stopping, checkpoint]
    )

# Build the best model
model = tuner.hypermodel.build(tuner.get_best_hyperparameters(1)[0])

# Retrain the model with original callbacks and data
start_time = time.time()  # Start timing
if y_val is not None:
    history = model.fit(
        X_train_proc, y_enc,
        validation_data=(X_val_proc, y_val),
        epochs=ep, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
else:
    history = model.fit(
        X_train_proc, y_enc,
        validation_split=0.2,
        epochs=ep, batch_size=bs,
        callbacks=[early_stopping, checkpoint],
        verbose=2
    )
end_time = time.time()  # End timing
duration = end_time - start_time  # Calculate duration

# Logging results
results = {
    'training_accuracy': history.history['accuracy'][-1],
    'training_loss': history.history['loss'][-1],
    'validation_accuracy': history.history['val_accuracy'][-1],
    'validation_loss': history.history['val_loss'][-1],
    'training_duration_seconds': duration
}
with open('results.json', 'w') as f:
    json.dump(results, f)

# Prediction and submission
raw_preds = model.predict(X_test_proc)
probs = raw_preds.flatten()
final_preds = (probs > 0.5).astype(int)

# Build submission
submission = pd.DataFrame({id_col: test_ids.reset_index(drop=True), target_col: final_preds})
submission.to_csv('submission_result.csv', index=False)

Trial 10 Complete [00h 04m 00s]
val_loss: 0.1520763337612152

Best val_loss So Far: 0.15107053518295288
Total elapsed time: 00h 44m 06s
Epoch 1/20


3518/3518 - 18s - 5ms/step - accuracy: 0.9317 - auc: 0.9677 - loss: 0.1690 - val_accuracy: 0.9397 - val_auc: 0.9749 - val_loss: 0.1516
Epoch 2/20
3518/3518 - 13s - 4ms/step - accuracy: 0.9364 - auc: 0.9703 - loss: 0.1612 - val_accuracy: 0.9391 - val_auc: 0.9750 - val_loss: 0.1517
Epoch 3/20


3518/3518 - 13s - 4ms/step - accuracy: 0.9369 - auc: 0.9707 - loss: 0.1602 - val_accuracy: 0.9400 - val_auc: 0.9750 - val_loss: 0.1513
Epoch 4/20


3518/3518 - 13s - 4ms/step - accuracy: 0.9367 - auc: 0.9709 - loss: 0.1593 - val_accuracy: 0.9402 - val_auc: 0.9751 - val_loss: 0.1512
Epoch 5/20
3518/3518 - 14s - 4ms/step - accuracy: 0.9368 - auc: 0.9712 - loss: 0.1590 - val_accuracy: 0.9399 - val_auc: 0.9751 - val_loss: 0.1515
Epoch 6/20
3518/3518 - 13s - 4ms/step - accuracy: 0.9374 - auc: 0.9714 - loss: 0.1589 - val_accuracy: 0.9399 - val_auc: 0.9750 - val_loss: 0.1518
Epoch 7/20
3518/3518 - 12s - 4ms/step - accuracy: 0.9371 - auc: 0.9711 - loss: 0.1590 - val_accuracy: 0.9394 - val_auc: 0.9748 - val_loss: 0.1522
Epoch 8/20
3518/3518 - 12s - 3ms/step - accuracy: 0.9376 - auc: 0.9715 - loss: 0.1583 - val_accuracy: 0.9394 - val_auc: 0.9748 - val_loss: 0.1518
Epoch 9/20
3518/3518 - 12s - 4ms/step - accuracy: 0.9373 - auc: 0.9717 - loss: 0.1582 - val_accuracy: 0.9397 - val_auc: 0.9749 - val_loss: 0.1520
Epoch 10/20
3518/3518 - 13s - 4ms/step - accuracy: 0.9379 - auc: 0.9715 - loss: 0.1582 - val_accuracy: 0.9398 - val_auc: 0.9749 - val_l

2025-07-15 22:46:49.568984: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16', 288 bytes spill stores, 288 bytes spill loads

2025-07-15 22:46:49.587631: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16', 48 bytes spill stores, 56 bytes spill loads

2025-07-15 22:46:49.646527: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16', 48 bytes spill stores, 56 bytes spill loads

2025-07-15 22:46:49.878442: I external/local_xla/xla/stream_executor/cuda/subprocess_compilation.cc:346] ptxas warning : Registers are spilled to local memory in function 'gemm_fusion_dot_16', 748 bytes spill stores, 748 bytes spill loads

2025-07-15 22:46:50.000843: I external/local

2932/2932 ━━━━━━━━━━━━━━━━━━━━ 7s 2ms/step


In [ ]:
print(duration)

188.128700017929
